Στην παρούσα εργασία, επέλεξα να ασχοληθώ με το μείζον θέμα στην Ελλάδα, τις πυρκαγιές που πλήττουν την χώρα ιδίως την περίοδο του καλοκαιριού. Έχοντας ως πρόσφατο και τραγικό παράδειγμα τις πυρκαγιές στην Εύβοια και στην Αττική το καλοκαίρι του 2021, απόφάσισα να ασχοληθώ με το θέμα με έναν διαφορετικό τρόπο.
Πιο συγκεκριμένα, θα γίνει συλλογή των τελευταίων 3.500 tweets από την πυροσβεστική υπηρεσία. 
Αξίζει να σημειωθεί, πως η πυροσβεστική υπηρεσία "ανεβάζει" κάποιο Tweet την στιγμή που εκδηλώνεται μία φωτιά ή διαδραματίζεται ένα σημαντικό γεγονός στο οποίο αναλαμβάνει δράση. Εκτός από τις πυρκαγιές, το πυροσβεστικό σώμα ενημερώνει το ελληνικό κοινό (μέσω του Twitter) για σοβαρά τροχαία, διασώσεις, εξαφανίσεις κ.α. Οπότε στην παρούσα έρευνα, θα υπάρξει μία προσπάθεια αναζήτησης των γεγονότων αυτών (με επίκεντρο τις φωτιές) μέσω αναζήτησης των λέξεων κλειδιών, η καταγραφή τους και η κατανομή τους σε πίνακα (dataframe), καθώς και ένα sentiment analysis των tweets αυτών. Το παρόν αρχείο θα μου χρησιμεύσει ώς την μηχανή αναζήτησς και συλλογής των tweets, Καθώς και τον καθαρισμό τους (scraping).

In [1]:
#Αρχικά θα εγκαταστήσουμε τις βιβλιοθήκες που θα μας χρειαστούν
!pip install Twython
import pandas as pd
import numpy as np 
from datetime import date
import os
from pathlib import Path
from google.colab import drive 
drive.mount('gdrive', force_remount=True)
from twython import Twython, TwythonError

CONSUMER_KEY = "Uv3Ccd0oSCf9LgawJx12EAqYl"

CONSUMER_SECRET = "NdsEcNJvTIhBVD8ApxthiidqxuY8MdKzmzPyb1tLdWCSDvLhW4"

OAUTH_TOKEN = "469275739-tpD9kkY4CCl2r8IZ2CimxNuF9DiEGtdb0fKwWLBK"

OAUTH_TOKEN_SECRET = "3Fx1ECS8mv4feMg6vfHsnFTGE5ExV845QHLPIgHuZOKF4"

twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at gdrive


In [2]:
twitter_user = "pyrosvestiki"

In [3]:
# Make twitter object with My keys and tokens
twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

# Get first 100 tweets
tweetsL = []
try:
    user_timeline = twitter.get_user_timeline(screen_name=twitter_user,count=100, tweet_mode = 'extended'  )
except TwythonError as e:
    print("Error getting tweets:", e)
print("I got:", len(user_timeline), " tweets")
for tweet in user_timeline:
    # Add whatever you want from the tweet, here we just add the text
    tweetsL.append(tweet)


# Get more 100s of tweets

# Count could be less than 200, see:
# https://dev.twitter.com/discussions/7513
while len(user_timeline) != 0: 
    try:
        user_timeline = twitter.get_user_timeline(screen_name=twitter_user, count=100,
                                                  tweet_mode = 'extended', max_id=user_timeline[len(user_timeline)-1]['id']-1)
    except TwythonError as e:
        print("Error getting tweets:", e)
    if len(user_timeline) > 0:
        print("I got:", len(user_timeline), " tweets more... Last ID:", user_timeline[len(user_timeline)-1]['id']-1)
    for tweet in user_timeline:
        # Add whatever you want from the tweet, here we just add the text
        tweetsL.append(tweet)
# Number of tweets the user has made
print("We got :", len(tweetsL), " tweets in total!!!")

I got: 100  tweets
I got: 100  tweets more... Last ID: 1540631155703177215
I got: 100  tweets more... Last ID: 1538072074194931713
I got: 100  tweets more... Last ID: 1534948050526928895
I got: 100  tweets more... Last ID: 1532122539639689223
I got: 100  tweets more... Last ID: 1528337398475673599
I got: 100  tweets more... Last ID: 1524674800576344063
I got: 100  tweets more... Last ID: 1520078644483350528
I got: 100  tweets more... Last ID: 1515549467604393983
I got: 100  tweets more... Last ID: 1511948943810338816
I got: 100  tweets more... Last ID: 1508298537477578759
I got: 100  tweets more... Last ID: 1504621302971355139
I got: 100  tweets more... Last ID: 1500432339888914432
I got: 100  tweets more... Last ID: 1495965592536428543
I got: 100  tweets more... Last ID: 1491723742195634184
I got: 100  tweets more... Last ID: 1487456537954115589
I got: 100  tweets more... Last ID: 1483362693503401984
I got: 100  tweets more... Last ID: 1478881611370446848
I got: 100  tweets more... La

Βλέπουμε, πως δούλεψε η διαδικασία συλλογής των Tweets του συγκεκριμένου λογαριασμού (σε αυτήν την περίπτωση της πυροσβεστικής υπηρεσίας). Ως επόμενο βήμα, ας δούμε τa tweets που συλλέξαμε, ώστε να είμαστε σίγουροι για την επιτυχία της διαδικασίας.

In [4]:
tweetsL[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Jul 07 16:56:59 +0000 2022',
 'display_text_range': [0, 122],
 'entities': {'hashtags': [{'indices': [27, 36], 'text': 'πυρκαγιά'}],
  'symbols': [],
  'urls': [],
  'user_mentions': []},
 'favorite_count': 10,
 'favorited': False,
 'full_text': 'Υπό μερικό έλεγχο τέθηκε η #πυρκαγιά σε χαμηλή βλάστηση σε υπαίθριο χώρο, πλησίον της οδού Ευκαλύπτων στο Μαρούσι Αττικής.',
 'geo': None,
 'id': 1545089506260537347,
 'id_str': '1545089506260537347',
 'in_reply_to_screen_name': 'pyrosvestiki',
 'in_reply_to_status_id': 1545074851496562691,
 'in_reply_to_status_id_str': '1545074851496562691',
 'in_reply_to_user_id': 158003436,
 'in_reply_to_user_id_str': '158003436',
 'is_quote_status': False,
 'lang': 'el',
 'place': None,
 'retweet_count': 1,
 'retweeted': False,
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Mo

In [5]:
for tweet in tweetsL:
    print(tweet['created_at'], tweet['full_text']) #ας δούμε και την ημερομηνία των tweets, ώστε να έχουμε κάποια ιδέα για ποια χρονική περίοδο μελετάμε.

Thu Jul 07 16:56:59 +0000 2022 Υπό μερικό έλεγχο τέθηκε η #πυρκαγιά σε χαμηλή βλάστηση σε υπαίθριο χώρο, πλησίον της οδού Ευκαλύπτων στο Μαρούσι Αττικής.
Thu Jul 07 16:52:04 +0000 2022 Σε συνέχεια προηγούμενου μηνύματος, υπό μερικό έλεγχο τέθηκε η #πυρκαγιά σε δασική έκταση στο Νέο Πετρίτσι Σερρών.
Thu Jul 07 16:15:44 +0000 2022 Σε συνέχεια προηγούμενου μηνύματος, υπό μερικό έλεγχο τέθηκε η #πυρκαγιά σε χαμηλή βλάστηση στην περιοχή Κιάφα του Δήμου Σπάτων Αρτέμιδος.
Thu Jul 07 16:01:08 +0000 2022 #Πυρκαγιά🔥σε χαμηλή βλάστηση στην περιοχή Κιάφα του Δήμου Σπάτων Αρτέμιδος. Επιχειρούν 18 #πυροσβέστες με 7 οχήματα.
Thu Jul 07 15:58:45 +0000 2022 #Πυρκαγιά σε χαμηλή βλάστηση σε υπαίθριο χώρο, πλησίον της οδού Ευκαλύπτων στο Μαρούσι Αττικής. Επιχειρούν 12 #πυροσβέστες με 5 οχήματα.
Thu Jul 07 15:06:50 +0000 2022 🚨49 δασικές πυρκαγιές 🔥 εκδηλώθηκαν το τελευταίο 24ωρο.
🔗https://t.co/sJioBK2IfS https://t.co/mo7V13vvIa
Thu Jul 07 13:41:00 +0000 2022 Κατεσβέσθη #πυρκαγιά σε Ε.Ι.Χ. αυτοκίνητο, επί 

Βλέπουμε πως συλλέχθηκαν με επιτυχία πάνω από 3.000 tweets. ΤΟ αξιοσημείωτο είναι πως το παλαιότερο tweet που συλλέξαμε είναι την Κυριακή 25 Ιουλίου 2021, κάτι που σημαίνει πως στην λίστα εντάσσεται και η περίοδος των μεγάλων πυρκαγιών σε Εύβοια και Αττική οι οποίες ξέσπασαν στις αρχές Αυγούστου.

Τώρα, αφού έχουμε σιγουρευτεί πως έχουμε μαζέψει τα σωστά tweets και τον αριθμό που θέλουμε, ας δούμε τα δεδομένα σε πίνακα και ας φτιάξουμε το dataframe μας.

In [6]:
tweets_df = pd.DataFrame(tweetsL)
tweets_df.head(1)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink
0,Thu Jul 07 16:56:59 +0000 2022,1545089506260537347,1545089506260537347,Υπό μερικό έλεγχο τέθηκε η #πυρκαγιά σε χαμηλή...,False,"[0, 122]","{'hashtags': [{'text': 'πυρκαγιά', 'indices': ...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.545075e+18,1545074851496562691,...,10,False,False,el,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
tweets_df.tail()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink
3245,Mon Aug 02 16:27:45 +0000 2021,1422232661767110667,1422232661767110667,Ολοκληρώθηκε #απεγκλωβισμός ενός ατόμου από Ε....,False,"[0, 183]","{'hashtags': [{'text': 'απεγκλωβισμός', 'indic...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,45,False,False,el,NaN,NaN,NaN,NaN,NaN,NaN
3246,Mon Aug 02 16:08:50 +0000 2021,1422227902033235969,1422227902033235969,5⃣2⃣ δασικές #πυρκαγιές 🔥 εκδηλώθηκαν το τελευ...,False,"[0, 105]","{'hashtags': [{'text': 'πυρκαγιές', 'indices':...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,48,False,False,el,"{'media': [{'id': 1422227871993671680, 'id_str...",False,NaN,NaN,NaN,NaN
3247,Mon Aug 02 14:49:30 +0000 2021,1422207936793911302,1422207936793911302,Στην δασική #πυρκαγιά στο Βασιλίτσι Μεσσηνίας ...,False,"[0, 208]","{'hashtags': [{'text': 'πυρκαγιά', 'indices': ...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.422190e+18,1422190042093608960,...,41,False,False,el,NaN,NaN,NaN,NaN,NaN,NaN
3248,Mon Aug 02 13:38:23 +0000 2021,1422190042093608960,1422190042093608960,#Πυρκαγιά σε δασική έκταση στην περιοχή Βασιλί...,False,"[0, 108]","{'hashtags': [{'text': 'Πυρκαγιά', 'indices': ...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,84,False,False,el,NaN,NaN,NaN,NaN,NaN,NaN
3249,Mon Aug 02 12:28:01 +0000 2021,1422172331527528454,1422172331527528454,Στην αγροτοδασική #πυρκαγιά στην περιοχή Λαγυν...,False,"[0, 213]","{'hashtags': [{'text': 'πυρκαγιά', 'indices': ...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.422142e+18,1422142424827240449,...,30,False,False,el,NaN,NaN,NaN,NaN,NaN,NaN


Αφού συλλέξαμε με επιτυχία τα tweets που μας ενδιαφέρουν, τα αποθηκεύω στο πρωσοπικό google drive.

In [8]:
file_name = "tweets_pyrosvestiki"

In [9]:
from google.colab import drive 
drive.mount('gdrive', force_remount=True)

Mounted at gdrive


In [10]:
tweets_df.to_csv("/content/gdrive/My Drive/Colab Notebooks/pyrosvestiki_tweets.tsv", sep='\t')